# Papers

## Abstract
-----
  Our unified architecture is extremely fast. Our base YOLO model processes images in real-time at 45 frames per second. A smaller version of the network, Fast YOLO, processes an astounding 155 frames per second while still achieving double the mAP of other real-time detectors. 

  Compared to state-of-the-art detection systems, YOLO makes more localization errors but is less likely to predict false positives on background. 

Finally, YOLO learns very general representations of objects. It outperforms other detection methods, including DPM and R-CNN, when gener- alizing from natural images to other domains like artwork.

<img alt="Figure 1: The YOLO Detection System" src="https://miro.medium.com/max/1400/1*YXP2MUhtyAo6bRWIFm1YAg.png"/>

<li>resizes the input image to 448 * 448</li>
<li>runs a single convolutional network on the image</li>
<li>thresholds the resulting detections by the model's sonfidence</li>

## 3 points should check(YOLOv1)
-----

  1. 빠른 처리 속도. 회귀 문제로, 복잡한 파이프라인을 작성할 필요가 없음. 이미지 처리 속도는 초당 45프레임(no batch processing, Titan X GPU). fast version(GPU로 봐야하는지, 최근 YOLO*모델로 해석해야하는지 보류)은 150 fps*를 넘음. 이는 스트리밍 비디오에서 처리 지연 속도는 25ms 미만이란 뜻.
$$YOLO \geqq 2*mAP(\mathrm{other\,real{-}time\,systems})$$
[their demo project webpage](https://pjreddie.com/darknet/yolo/)

  2. 높은 예측 설득력. (sliding window와 region proposal-based techniques와 다르게) YOLO는 train과 test에서의 전체 이미지를 보기 때문에, 즉시 class별 특성을 통해 문맥상 정보(contextual information)를 인코딩한다. 탐지 방법 중 최고인 Fast R-CNN*(2016년 기준), 전체 이미지(the larger context)를 볼 수 없기 때문에, 이미지의 객체도 배경으로 처리하는(patches) 실수가 있다.

$\mathrm{the\,number\,of\,background\,errors,}$
$$YOLO\leqq\frac{Fast R-CNN}{2}$$

  3. 일반화 할 수 있는(generalizable) 대표 특성 학습. wide margin으로 DPM이나 R-CNN과 같은 상위 탐지 모델을 능가함. new domains나 unexpected inputs을 적용해도, 일반화 가능성이 높기 때문에 거의 깨지지 않음.

</br>
  YOLO(v1 기준)는 여전히 최신 기술(state-of-the-art) 중 정확도 측면에서 뒤쳐짐. 이미지 내 객체를 빨리 구별이 가능하지만, 특히 작은 객체 등의 몇몇 객체를 정확하게 초점을 맞추는 것(localize)은 어려워 함. training, testing 코드는 오픈소스이며, 여러 pretrained models도 다운 가능하다고 적혀있음.
</br></br></br>
<p>
* fps : frames per second</br>
* YOLO : You Only Look Once</br>
* Fast R-CNN : Fast Regions with CNN Features</br>
* DPM : Deformable parts models</br>
* R-CNN : Regions with CNN Features</br>
</p>

## Unified Detection
-----



<p align="center"><img width="500" alt="The Model- YOLOv1" src="https://pjreddie.com/media/image/model_2.png"/></p>

* divides the image into an S × S grid
* predicts B bounding boxes for each grid cell
* predicts confidence for those boxes, and C class probabilities <- be encoded as an S × S × (B ∗ 5 + C) tensor

[//]:<> (단일 neural network에서 객체 탐지 요소를 분할, 전체 이미지에서 얻은 특성을 사용해 각각의 bounding box 에측, 한 이미지의 유사도를 구하기 위해 모든 클래스_훈련+검증/실시간_ 아우르는 모든 bounding box를 예측, 높은 정확성을 유지하면서도 훈련 시작부터 실시간 데이터 속도까지 사용가능하도록 설계됨)

[//]: <> (input image를 간격을 S로 해서 나눔. 객체의 중점이 하나의 격자 공간_grid cell_에 있다면, 객체 탐지가 가능함. 각각의 격자 공간은 B bounding boxes와 박스의 신뢰도?_confidence scores stands for?_를 예측. 이 신뢰도는 객체를 박스에 포함하는 모델이 어떻게 설득력을 가지는지와 예측한 박스로 도출한 것을 정확히 할 것인지를 반영)

$\mathrm{Confidence} = \mathrm{Pr}(Object) * IOU_{pred}^{truth}, $

</br>

$$\begin{cases}\mathrm{Confidence} = 0\;\;\;If object exists in that cell\\\\\mathrm{Confidence} = IOU\;\;\;between\,the\,predicted\,box\,and\,the\,ground\,truth\end{cases}$$

</br>

**5 predictions for each BBox**
- (x, y) : 격자 공간의 가장자리와 관련이 높은 박스의 중점
- (w, h) : 전체 이미지 중 관련도가 높다고 예측된 가로, 세로값
- c(confidence) : $IOU_{pred}^{truth}$

[//]:<> (각각의 격자 공간은 $\mathrm{Pr}\lgroup{Class_i|Object}\rgroup$_클래스별 조건부 확률_도 예측, 이 확율이 격자 공간이 객체를 포함하는지에 대한 조건이 됨_weights나 feature information이라고 봐도 되는 부분?_. 격자 공간당 클래스별 확률은 하나씩만 예측가능._BBox수랑 무관_. 검증 시 클래스별 조건부확률 * 개별 box 얘상 정확도?)

</br>

$$\mathrm{Pr}(Class_i|Object) * \mathrm{Pr}(Object) * IOU_{pred}^{truth} = \mathrm{Pr}(Class_i) * IOU_{pred}^{truth}$$

[//]:<> (위의 수식은 박스별 특정 클래스 신뢰도?를 구함. 그 점수는 박스 내 클래스의 확률과 해당 객체와 예측 박스의 적합도로 인코딩됨.)

</br>

$\small\mathrm{For\,evaluating\,YOLOv1\,on\,PASCAL\,VOC},\\\\\begin{align*} \\\large S = 7, \\\large B = 2, \\\large C = 20\,\small(20\,labelled\,classes), \\\large final\,prediction = 7 * 7 * 30\, \small tensor \end{align*}$

</br>
<p>
* IOU : intersection over union
</p>



<p align="center"><img width="500" alt="The Architecture" src="https://production-media.paperswithcode.com/methods/Screen_Shot_2020-06-24_at_12.22.30_PM.png"/></p>

[//]:<> (Our detection network has 24 convolutional layers followed by 2 fully connected layers. Alternating 1 × 1 convolutional layers reduce the features space from preceding layers. We pretrain the convolutional layers on the ImageNet classification task at half the resolution_224 × 224 input image_ and then double the resolution for detection.)



## Question Mark

주최 측이 평가한다던, $macro\,F1\,Score$가 class probability map이랑 유사한 원리일 것이라 생각됨.

# Codes

```shell
// ~.sh
%%bash
!wget 
// toml or yml -> command line
// 작성해야할 리눅스 명령어는 도커 빌드하면서 연습해보고 정리하기 <- p.s. colab에는 거의 셋업이 되어 있음
```

```python
# import ROOT 

# cppFunctionCode = '''
# void f() {
# std::cout << "Hi jitted C. world!" « std::endl; 
# }'''

# ROOT.gInterpreter.Declare(cppFunctionCode) 
# ROOT.f() # Hello! 


##### python documents #####
# parser
# functools
# turtle
# pickle
# json
# io
# pathlib
# smath
# curses
# gc
# ensurepip
```

In [ ]:
# 환경, 경로 등
import os
import sys
import shutil
from glob import glob as gb
# from setuptools import setup, Extension # def, class 작성 이후

# 비동기, 속도 등
import tqdm
# import time
import timeit
# timeit.timeit(~), timeit.Timer(~)
import numba
import asyncio

# 이미지
import base64
import numpy as np
import cv2 as cv

# 모델
import torch

# 결과 도출
# import json
from flask import json


In [ ]:
# 경로 - colab


# 시각화 - 모델 조율, 더블 체크 등에 활용

In [ ]:
CONFIDENCE = 0.5
CONFIG = "yolov3.cfg"
WEIGHTS = "yolov3.weights"
LABELS = "coco.names"

with open(LABELS, 'r') as f:
    labels = f.read().strip().split('\n')

net = cv2.dnn.readNetFromDartnet(CONFIG, WEIGHTS)
# 물론 외부 네트워크 제한으로, 해당 모듈 사용 불가함
# 미리 다운받을 수 있도록 commands로 접근했을 때, 경로 확인 필수

def coco_process(ecdImage):
    image_64_decode = base64.decodebytes(encodedImage.encode('utf-8'))

    with 